In [9]:
from google.colab import drive
drive.mount('/content/drive')

# 1. 라이브러리 및 경로 설정
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Google Drive 경로 (필요에 맞게 수정)
BASE_PATH = "/content/drive/MyDrive/ML_Dataset"

# 2. Feature 파일 불러오기
X_burst_mon = np.load(f"{BASE_PATH}/features_burst_mon.npy")
X_cum_mon   = np.load(f"{BASE_PATH}/features_cum_mon.npy")
X_inout_mon = np.load(f"{BASE_PATH}/features_inout_mon.npy")

X_burst_unmon = np.load(f"{BASE_PATH}/features_burst_unmon.npy")
X_cum_unmon   = np.load(f"{BASE_PATH}/features_cum_unmon.npy")
X_inout_unmon = np.load(f"{BASE_PATH}/features_inout_unmon.npy")

print("Loaded all feature files successfully!\n")
print("Monitored shapes:", X_burst_mon.shape, X_cum_mon.shape, X_inout_mon.shape)
print("Unmonitored shapes:", X_burst_unmon.shape, X_cum_unmon.shape, X_inout_unmon.shape)

# 3. Feature 병합 (열 기준)
X_mon = np.concatenate([X_burst_mon, X_cum_mon, X_inout_mon], axis=1)
X_unmon = np.concatenate([X_burst_unmon, X_cum_unmon, X_inout_unmon], axis=1)

print(f"\nCombined Monitored shape: {X_mon.shape}")
print(f"Combined Unmonitored shape: {X_unmon.shape}")

# 4. Label 생성
# Monitored: 95개 클래스 (0~94)
# Unmonitored: -1 label
y_mon = np.repeat(np.arange(95), 200)
y_unmon = np.full(X_unmon.shape[0], -1)

print(f"\nMonitored labels shape: {y_mon.shape}")
print(f"Unmonitored labels shape: {y_unmon.shape}")

# 5. Open-World용 전체 데이터 결합
X_total = np.concatenate([X_mon, X_unmon], axis=0)
y_total = np.concatenate([y_mon, y_unmon], axis=0)

print("\nTotal dataset shape:", X_total.shape)
print("Total label shape:", y_total.shape)

# 6. Train/Test Data Split
# Stratified sampling은 다중클래스에만 가능하므로, open-world binary 설정으로 분할
y_binary = np.where(y_total == -1, 0, 1)  # monitored=1, unmonitored=0

X_train, X_test, y_train, y_test = train_test_split(
    X_total, y_binary, test_size=0.25, random_state=42, stratify=y_binary
)

print(f"\nTrain shape: {X_train.shape}, Test shape: {X_test.shape}")

# 7. Feature Scaling (표준화)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\nFeature scaling completed!")


# 8. Save
np.save(f"{BASE_PATH}/X_train_scaled.npy", X_train_scaled)
np.save(f"{BASE_PATH}/X_test_scaled.npy", X_test_scaled)
np.save(f"{BASE_PATH}/y_train.npy", y_train)
np.save(f"{BASE_PATH}/y_test.npy", y_test)

print("Saved all processed data! Ready for model training.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded all feature files successfully!

Monitored shapes: (19000, 11) (19000, 10) (19000, 4)
Unmonitored shapes: (3000, 11) (3000, 10) (3000, 4)

Combined Monitored shape: (19000, 25)
Combined Unmonitored shape: (3000, 25)

Monitored labels shape: (19000,)
Unmonitored labels shape: (3000,)

Total dataset shape: (22000, 25)
Total label shape: (22000,)

Train shape: (16500, 25), Test shape: (5500, 25)

Feature scaling completed!
Saved all processed data! Ready for model training.


In [10]:
files = {
    "X_train_scaled": f"{BASE_PATH}/X_train_scaled.npy",
    "X_test_scaled": f"{BASE_PATH}/X_test_scaled.npy",
    "y_train": f"{BASE_PATH}/y_train.npy",
    "y_test": f"{BASE_PATH}/y_test.npy",
}

for name, path in files.items():
    arr = np.load(path)
    print(f"\n📁 {name}")
    print(f"Shape: {arr.shape}")
    print(f"Dtype: {arr.dtype}")

    # 숫자형 배열만 통계 출력
    if np.issubdtype(arr.dtype, np.number):
        print(f"Mean: {np.mean(arr):.4f}, Std: {np.std(arr):.4f}")
        print(f"Min: {np.min(arr):.4f}, Max: {np.max(arr):.4f}")

    print("-" * 60)

y_train = np.load(files["y_train"])
y_test = np.load(files["y_test"])

unique_train, counts_train = np.unique(y_train, return_counts=True)
unique_test, counts_test = np.unique(y_test, return_counts=True)

print("\n Label Distribution (Train):")
for u, c in zip(unique_train, counts_train):
    print(f"  Class {int(u)}: {c} samples")

print("\n Label Distribution (Test):")
for u, c in zip(unique_test, counts_test):
    print(f"  Class {int(u)}: {c} samples")

print("\n Dataset integrity check complete!")



📁 X_train_scaled
Shape: (16500, 25)
Dtype: float64
Mean: 0.0000, Std: 0.9592
Min: -16.3972, Max: 128.4484
------------------------------------------------------------

📁 X_test_scaled
Shape: (5500, 25)
Dtype: float64
Mean: 0.0010, Std: 1.0114
Min: -16.3972, Max: 173.9198
------------------------------------------------------------

📁 y_train
Shape: (16500,)
Dtype: int64
Mean: 0.8636, Std: 0.3432
Min: 0.0000, Max: 1.0000
------------------------------------------------------------

📁 y_test
Shape: (5500,)
Dtype: int64
Mean: 0.8636, Std: 0.3432
Min: 0.0000, Max: 1.0000
------------------------------------------------------------

 Label Distribution (Train):
  Class 0: 2250 samples
  Class 1: 14250 samples

 Label Distribution (Test):
  Class 0: 750 samples
  Class 1: 4750 samples

 Dataset integrity check complete!
